In [623]:
import pandas as pd
import matplotlib.pyplot as plt

### Read data to DataFrames
#### Read data file project_export.csv

In [460]:
# default na_values is ''
project_ex = pd.read_csv('../data/project_export.csv')

#### Read data file session_export.csv

In [461]:
session_ex = pd.read_csv('../data/session_export.csv', na_values=['','{}'])

### Process DataFrames

#### Processing data frame - session_ex. Output DataFrame - session

In [462]:
session_ex.info()

last_element = lambda x: x.split(',')[-1].strip('"}').strip('"').strip('{"')
session_ex[session_ex['time_slots'].isnull()].shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1665 entries, 0 to 1664
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       1665 non-null   int64  
 1   created_at               1665 non-null   object 
 2   updated_at               1665 non-null   object 
 3   state                    1665 non-null   object 
 4   description              1664 non-null   object 
 5   consultant_id            623 non-null    float64
 6   nonprofit_id             1665 non-null   int64  
 7   scheduled_for            576 non-null    object 
 8   time_slots               1622 non-null   object 
 9   organization_id          1665 non-null   int64  
 10  conference_line_id       625 non-null    float64
 11  project_category_id      1660 non-null   float64
 12  partner_organization_id  0 non-null      float64
 13  archived                 1665 non-null   object 
dtypes: float64(4), int64(3),

(43, 14)

- convert created_at and updated_at to datetime type

In [463]:
session_ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1665 entries, 0 to 1664
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       1665 non-null   int64  
 1   created_at               1665 non-null   object 
 2   updated_at               1665 non-null   object 
 3   state                    1665 non-null   object 
 4   description              1664 non-null   object 
 5   consultant_id            623 non-null    float64
 6   nonprofit_id             1665 non-null   int64  
 7   scheduled_for            576 non-null    object 
 8   time_slots               1622 non-null   object 
 9   organization_id          1665 non-null   int64  
 10  conference_line_id       625 non-null    float64
 11  project_category_id      1660 non-null   float64
 12  partner_organization_id  0 non-null      float64
 13  archived                 1665 non-null   object 
dtypes: float64(4), int64(3),

- Drop irrelevant columns in session - 'description', 'consultant_id', 'conference_line_id', 'partner_organization_id', 'archived'

In [464]:
drop_col = ['description', 'consultant_id', 'conference_line_id', 'partner_organization_id', 'archived']
session = session_ex.drop(drop_col, axis=1)
session.info()
#session['scheduled_for']
session_ex.scheduled_for

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1665 entries, 0 to 1664
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   1665 non-null   int64  
 1   created_at           1665 non-null   object 
 2   updated_at           1665 non-null   object 
 3   state                1665 non-null   object 
 4   nonprofit_id         1665 non-null   int64  
 5   scheduled_for        576 non-null    object 
 6   time_slots           1622 non-null   object 
 7   organization_id      1665 non-null   int64  
 8   project_category_id  1660 non-null   float64
dtypes: float64(1), int64(3), object(5)
memory usage: 117.2+ KB


0                 NaN
1       3/31/17 16:00
2       3/31/16 19:00
3        9/6/16 13:00
4       3/24/16 14:00
            ...      
1660              NaN
1661              NaN
1662              NaN
1663              NaN
1664    12/5/18 16:00
Name: scheduled_for, Length: 1665, dtype: object

- Find columns with missing values - 'scheduled_for', 'time_slots', 'project_category_id'

In [465]:
cols_na = session.columns[session.isnull().sum(axis=0) > 0]
cols_na

Index(['scheduled_for', 'time_slots', 'project_category_id'], dtype='object')

- Handle missing values in columns 'scheduled_for' and 'time_slots'

   \- Add today's datetime to mark the missing values in columns. Also change values in 'time_slots' to be the latest time it has.

In [466]:
# Set missing values in column 'scheduled_for' to be special text 'today' 
session.loc[session.scheduled_for.isnull(), 'scheduled_for'] = 'today'
# Set time_slots values to be the last element of column 'time_slots'
session.loc[session.time_slots.notnull(), 'time_slots'] = session.loc[session.time_slots.notnull()].time_slots.apply(last_element)
session.loc[session.time_slots.isnull(), 'time_slots'] = 'today'

- Handle missing values in column 'project_category_id'
   
   \- 1. Infer project_category_id according to the description
   
   \- 2. Delete record with id=365. Because description has meaningless characters that doesn't help to infer project_category

In [467]:
# Infer project_category_id according to the description
session.loc[session.id==348, 'project_category_id'] = 15
session.loc[session.id==402, 'project_category_id'] = 3
session.loc[session.id==412, 'project_category_id'] = 2
session.loc[session.id==421, 'project_category_id'] = 14
# Delete record with id=365. Because description has meaningless characters that doesn't help to infer project_category
session = session.drop(27, axis=0)

In [468]:
# set project_category_id to int64
session['project_category_id'] = pd.to_numeric(session.project_category_id, downcast='integer')

- Now DataFrame session has no missing values

- Convert columns - scheduled_for and time_slots to datetime type

In [ ]:
session['scheduled_for'] = session['scheduled_for'].apply(pd.to_datetime)
session['time_slots'] = session['time_slots'].apply(pd.to_datetime)

In [436]:
session.info()
#session.to_csv('../clean_data/session.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1664 entries, 0 to 1664
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   1664 non-null   int64 
 1   created_at           1664 non-null   object
 2   updated_at           1664 non-null   object
 3   state                1664 non-null   object
 4   nonprofit_id         1664 non-null   int64 
 5   scheduled_for        1664 non-null   object
 6   time_slots           1664 non-null   object
 7   organization_id      1664 non-null   int64 
 8   project_category_id  1664 non-null   int8  
dtypes: int64(3), int8(1), object(5)
memory usage: 118.6+ KB


#### Processing Data Frame - project_ex. Output DataFrame - project

- Check missing values and handle missing values

    \- All the columns with missing values are irrelevant, drop the columns

- Remove irrelevant columns - 'local_only', 'timeline', 'publish_externally', 'enable_success_story', 'is_archived'

In [235]:
# Check NA columns in dataFrame 'project_ex'
## Method_1
project_ex.info()
## Method_2 - get a list
col_na = [c for c in project_ex.columns if any(project_ex[c].isnull())]
## Method_3 - get an index object
col_na = project_ex.columns[project_ex.isnull().sum(axis=0) > 0]
# All the columns having missing value are irrelevant, so drop columns
project = project_ex.drop(col_na, axis=1)
# Also add to drop list with columns - local_only, timeline, publish_externally, enable_success_story, is_archived
col_na = ['local_only', 'timeline', 'publish_externally', 'enable_success_story', 'is_archived']
project = project.drop(col_na, axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11549 entries, 0 to 11548
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       11549 non-null  int64  
 1   organization_id          11549 non-null  int64  
 2   description              9874 non-null   object 
 3   created_at               11549 non-null  object 
 4   updated_at               11549 non-null  object 
 5   state                    11549 non-null  object 
 6   user_id                  11549 non-null  int64  
 7   needs_accomplish         11547 non-null  object 
 8   needs_support            10830 non-null  object 
 9   needs_value              10817 non-null  object 
 10  campaign_id              455 non-null    float64
 11  image_id                 11507 non-null  float64
 12  project_inquiries_count  11549 non-null  int64  
 13  admin_id                 9313 non-null   float64
 14  project_group         

- DataFrame project has no missing values now

- Convert strings to datetime for columns - created_at, updated_at

In [491]:
project[['created_at', 'updated_at']] = project[['created_at', 'updated_at']].apply(pd.to_datetime)

In [492]:
project.info()
#project.to_csv('../clean_data/project.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11549 entries, 0 to 11548
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       11549 non-null  int64         
 1   organization_id          11549 non-null  int64         
 2   created_at               11549 non-null  datetime64[ns]
 3   updated_at               11549 non-null  datetime64[ns]
 4   state                    11549 non-null  object        
 5   user_id                  11549 non-null  int64         
 6   project_inquiries_count  11549 non-null  int64         
 7   project_category_id      11549 non-null  int8          
dtypes: datetime64[ns](2), int64(4), int8(1), object(1)
memory usage: 643.0+ KB


### Data Exploration

In [493]:
project.head()

,id,organization_id,created_at,updated_at,state,user_id,project_inquiries_count,project_category_id
0,5948,4809,2017-07-06 17:27:44.905846,2017-08-30 01:39:47.519662,closed,148960,3,19
1,5908,2051,2017-06-28 16:55:47.852560,2017-11-11 18:17:38.379281,completed,122660,3,14
2,9339,1401,2019-01-15 00:00:34.636008,2019-01-15 00:00:34.636008,draft,169620,0,14
3,5975,4831,2017-07-10 15:36:19.184276,2017-09-27 13:39:17.766539,closed,149358,0,2
4,9981,7860,2019-04-03 03:19:57.848510,2019-04-03 03:19:57.848510,draft,172963,0,1


- Exploration of DataFrame 'project'

In [494]:
project.info()
project.describe()
print('\nTotal organizations with projects: ' + str(len(project['organization_id'].unique())))
print('All possible project states: ' + str(len(project['state'].unique())))
print('How many nonprofit users with projects: ' + str(len(project['user_id'].unique())))
print('Project categories in total: ' + str(len(project['project_category_id'].unique())))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11549 entries, 0 to 11548
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       11549 non-null  int64         
 1   organization_id          11549 non-null  int64         
 2   created_at               11549 non-null  datetime64[ns]
 3   updated_at               11549 non-null  datetime64[ns]
 4   state                    11549 non-null  object        
 5   user_id                  11549 non-null  int64         
 6   project_inquiries_count  11549 non-null  int64         
 7   project_category_id      11549 non-null  int8          
dtypes: datetime64[ns](2), int64(4), int8(1), object(1)
memory usage: 643.0+ KB

Total organizations with projects: 5152
All possible project states: 11
How many nonprofit users with projects: 5601
Project categories in total: 21


 \- Can one organization initiate multiple projects with different project categories?
 
   1. One organization could initiate multiple projects with same or different project category id(s)
   2. One organization could initiate multiple projects with same project category id by one or multiple person(s)
   
   So organization and project category id cannot uniquely identify one record in project
   

In [495]:
type(grp)

pandas.core.groupby.generic.DataFrameGroupBy

In [499]:
grp = project.groupby(['organization_id', 'user_id'])
cat_pergrp = grp.project_category_id.nunique()
cat_pergrp.sort_values(ascending=False)

# is organization_id and project_category_id identify a unique record
project.groupby(['organization_id', 'project_category_id'])['user_id'].nunique().sort_values(ascending=False)
(project.groupby(['organization_id', 'project_category_id'])['user_id'].nunique()>1).sum()
project.groupby(['organization_id', 'project_category_id'])['user_id'].count().sum()
(project.groupby(['organization_id', 'project_category_id'])['user_id'].count()>1).sum()


1449

   \- Distribution of projects by 'project_state' and 'project_category_id'

In [500]:
project.pivot_table(values='id', index='project_category_id', columns='state', 
                    aggfunc='count', fill_value=0, margins=True)

state,admin_close,applied,archived,closed,completed,deactivated,draft,matched,published,rejected,reviewable,All
project_category_id,,,,,,,,,,,,
1,2,6,9,314,211,12,8,34,20,78,35,729
2,2,9,5,429,373,18,10,35,49,154,44,1128
3,1,3,1,36,54,2,2,11,4,18,6,138
4,0,7,7,55,60,8,3,22,9,47,32,250
5,0,1,2,75,84,6,2,7,4,28,13,222
6,3,2,2,69,101,7,2,18,20,12,13,249
7,1,1,3,85,148,11,3,18,10,23,11,314
8,3,5,3,101,100,5,4,13,13,37,29,313
9,0,2,4,56,57,4,1,8,7,32,13,184


   \- Average inquries for each 'project_state' and 'project_category_id'

In [501]:
project.pivot_table(values='project_inquiries_count', index='project_category_id', columns='state',
                   aggfunc='mean', fill_value=0, margins=True)

state,admin_close,applied,archived,closed,completed,deactivated,draft,matched,published,rejected,reviewable,All
project_category_id,,,,,,,,,,,,
1,1.000000,0,1.666667,2.044586,2.909953,3.333333,0,2.411765,1.850000,0.000000,0,1.964335
2,2.000000,0,1.800000,2.179487,2.823056,3.111111,0,3.000000,1.918367,0.058442,0,2.007979
3,1.000000,0,1.000000,2.611111,2.907407,3.500000,0,3.454545,2.500000,0.000000,0,2.231884
4,0.000000,0,1.142857,3.818182,4.483333,9.375000,0,8.500000,5.000000,0.021277,0,3.180000
5,0.000000,0,0.000000,1.653333,3.404762,5.166667,0,5.714286,3.250000,0.000000,0,2.225225
6,5.000000,0,0.500000,1.826087,3.188119,5.428571,0,2.888889,2.500000,0.000000,0,2.425703
7,2.000000,0,3.000000,3.517647,4.635135,8.181818,0,3.777778,1.600000,0.130435,0,3.735669
8,1.666667,0,0.666667,2.415842,2.440000,1.400000,0,2.538462,1.692308,0.000000,0,1.779553
9,0.000000,0,0.750000,2.500000,3.684211,4.250000,0,6.875000,3.285714,0.000000,0,2.434783


- Exploration of dataFrame - session

In [502]:
session.info()
session.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1664 entries, 0 to 1664
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   1664 non-null   int64         
 1   created_at           1664 non-null   object        
 2   updated_at           1664 non-null   object        
 3   state                1664 non-null   object        
 4   nonprofit_id         1664 non-null   int64         
 5   scheduled_for        1664 non-null   datetime64[ns]
 6   time_slots           1664 non-null   datetime64[ns]
 7   organization_id      1664 non-null   int64         
 8   project_category_id  1664 non-null   int8          
dtypes: datetime64[ns](2), int64(3), int8(1), object(3)
memory usage: 118.6+ KB


,id,created_at,updated_at,state,nonprofit_id,scheduled_for,time_slots,organization_id,project_category_id
0,102,04:55.6,05:13.7,cancelled,2036,2021-07-09 22:21:38.182222,2017-01-20 19:00:00,239,20
1,110,36:50.9,00:00.4,completed,144136,2017-03-31 16:00:00.000000,2017-03-31 21:00:00,4218,20
2,60,08:47.4,52:12.5,cancelled,11682,2016-03-31 19:00:00.000000,2016-04-01 16:00:00,2153,20
3,95,37:31.0,00:01.8,completed,131196,2016-09-06 13:00:00.000000,2016-09-06 13:00:00,2933,10
4,53,05:42.5,00:01.9,completed,9367,2016-03-24 14:00:00.000000,2016-03-25 15:00:00,1370,20


   \- Session distribution by 'project_category_id' and session 'state'
   
   \- 1. Expired session has highest perentage among all sessions

In [503]:
session.pivot_table(values='id', columns='state', index='project_category_id',
                   aggfunc='count', fill_value=0, margins=True)

state,applied,cancelled,completed,draft,expired,matched,missed,npo_rescheduled,pbc_rescheduled,pending,published,All
project_category_id,,,,,,,,,,,,
1,9,18,41,26,68,0,4,3,0,20,1,190
2,8,19,70,26,57,0,1,0,4,12,1,198
3,1,0,11,3,10,0,1,0,0,2,0,28
4,0,4,9,7,4,0,0,0,0,3,1,28
5,3,1,9,10,17,2,0,1,1,4,1,49
6,0,4,5,2,10,0,0,2,0,1,0,24
7,0,6,13,8,22,0,0,0,0,2,2,53
8,2,9,28,14,28,0,2,2,2,8,0,95
9,4,1,10,5,11,0,0,0,0,3,0,34


   \- 2. All expired session has no scheduled time.

   \- 67/185 sessions with cancelled state has scheduled times
   
   \- All sessions with state of missed has scheduled times

In [537]:
session.loc[(session.state=='expired') & (session.scheduled_for.dt.date != pd.to_datetime('today').date())]
session.loc[(session.state=='cancelled')&(session.scheduled_for.dt.date != pd.to_datetime('today').date())]
session.loc[(session.state=='missed')&(session.scheduled_for.dt.date == pd.to_datetime('today').date())]

,id,created_at,updated_at,state,nonprofit_id,scheduled_for,time_slots,organization_id,project_category_id


   \- One session with completed state has missing sheduled time

In [538]:
session.loc[(session.state=='completed')&(session.scheduled_for.dt.date==pd.to_datetime('today').date())]

,id,created_at,updated_at,state,nonprofit_id,scheduled_for,time_slots,organization_id,project_category_id
604,635,34:42.6,06:43.2,completed,165441,2021-07-09 22:21:38.371462,2018-10-19 16:00:00,2422,10


  \- Filter out normal sessions for use

In [539]:
session_normal = session.loc[~session.state.isin(['expired', 'cancelled', 'missed'])]
session_normal.info()
session_normal.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 913 entries, 1 to 1663
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   913 non-null    int64         
 1   created_at           913 non-null    object        
 2   updated_at           913 non-null    object        
 3   state                913 non-null    object        
 4   nonprofit_id         913 non-null    int64         
 5   scheduled_for        913 non-null    datetime64[ns]
 6   time_slots           913 non-null    datetime64[ns]
 7   organization_id      913 non-null    int64         
 8   project_category_id  913 non-null    int8          
dtypes: datetime64[ns](2), int64(3), int8(1), object(3)
memory usage: 65.1+ KB


,id,created_at,updated_at,state,nonprofit_id,scheduled_for,time_slots,organization_id,project_category_id
1,110,36:50.9,00:00.4,completed,144136,2017-03-31 16:00:00,2017-03-31 21:00:00,4218,20
3,95,37:31.0,00:01.8,completed,131196,2016-09-06 13:00:00,2016-09-06 13:00:00,2933,10
4,53,05:42.5,00:01.9,completed,9367,2016-03-24 14:00:00,2016-03-25 15:00:00,1370,20
5,94,43:35.7,00:11.0,completed,131204,2016-08-23 20:00:00,2016-08-31 21:00:00,2934,10
6,76,14:55.1,00:02.0,completed,8231,2016-04-15 16:00:00,2016-04-15 16:00:00,1180,20


   \- How many organizations held normal sessions? 
   <br>\- How many normal sessions an organization held?

In [540]:
session_normal.groupby('organization_id')['id'].count().sort_values(ascending=False)

organization_id
8495    9
4291    6
8615    6
7574    6
8551    6
       ..
6437    1
6446    1
6459    1
6468    1
9329    1
Name: id, Length: 683, dtype: int64

In [541]:
session_normal.groupby('organization_id')['project_category_id'].nunique().sort_values(ascending=False)

organization_id
7574    6
7534    5
4291    5
5485    4
5127    4
       ..
6415    1
6416    1
6424    1
6431    1
9329    1
Name: project_category_id, Length: 683, dtype: int64

In [542]:
session.set_index(['state', 'project_category_id']).sort_index()

id created_at updated_at  nonprofit_id  \
state     project_category_id                                             
applied   1                     825    29:42.8    29:42.8        168584   
          1                    1650    59:38.5    59:38.5        180534   
          1                    1536    29:52.2    29:52.2        179794   
          1                    1537    30:02.0    30:02.0        179794   
          1                    1651    59:46.0    59:46.0        180534   
...                             ...        ...        ...           ...   
published 12                   1831    53:50.1    06:08.6        176320   
          12                   1889    52:23.1    12:23.2        183374   
          16                   1844    38:27.7    32:18.1        182127   
          16                   1845    38:35.5    32:18.1        182127   
          18                   1881    13:39.6    55:25.4        183319   

                                           scheduled_for          time_slots  \
state     project_category_id                                                  
applied   1                   2021-07-09 22:21:38.383319 2019-01-31 12:00:00   
          1                   2021-07-09 22:21:38.504923 2019-10-11 18:00:00   
          1                   2021-07-09 22:21:38.505172 2019-09-25 19:00:00   
          1                   2021-07-09 22:21:38.505407 2019-09-25 19:00:00   
          1                   2021-07-09 22:21:38.508256 2019-10-11 18:00:00   
...                                                  ...                 ...   
published 12                  2021-07-09 22:21:38.543203 2019-12-23 15:00:00   
          12                  2021-07-09 22:21:38.566489 2019-12-17 19:00:00   
          16                  2021-07-09 22:21:38.559019 2019-12-13 20:00:00   
          16                  2021-07-09 22:21:38.559134 2019-12-13 20:00:00   
          18                  2021-07-09 22:21:38.566035 2019-12-19 20:00:00   

                               organization_id  
state     project_category_id                   
applied   1                               7205  
          1                               8898  
          1                               8773  
          1                               8773  
          1                               8898  
...                                        ...  
published 12                              8265  
          12                              9282  
          16                              9111  
          16                              9111  
          18                              9275  

[1664 rows x 7 columns]

- Exploration of project and session combined

   \- Link dataFrame project and session by join operation. Create new dataFrame - project_session
    <br>  Join by 'organization_id' and 'project_category_id'. In output dataFrame, some records have project created before session scheduled time. We can drop these rows, because once project was created, no session is needed for the project. So the session shouldn't be linked to the project. 

In [543]:
session_normal.info()
project.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 913 entries, 1 to 1663
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   913 non-null    int64         
 1   created_at           913 non-null    object        
 2   updated_at           913 non-null    object        
 3   state                913 non-null    object        
 4   nonprofit_id         913 non-null    int64         
 5   scheduled_for        913 non-null    datetime64[ns]
 6   time_slots           913 non-null    datetime64[ns]
 7   organization_id      913 non-null    int64         
 8   project_category_id  913 non-null    int8          
dtypes: datetime64[ns](2), int64(3), int8(1), object(3)
memory usage: 65.1+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11549 entries, 0 to 11548
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------         

In [544]:
project_session = pd.merge(project, session_normal, on=['organization_id', 'project_category_id'], how='inner')
project_session.head()

,id_x,organization_id,created_at_x,updated_at_x,state_x,user_id,project_inquiries_count,project_category_id,id_y,created_at_y,updated_at_y,state_y,nonprofit_id,scheduled_for,time_slots
0,5129,3853,2017-03-14 13:30:52.150201,2017-07-31 18:24:42.461761,completed,140381,3,10,320,12:50.1,00:46.8,completed,140381,2018-02-14 19:00:00,2018-02-23 20:00:00
1,7449,5900,2018-02-16 19:26:51.490331,2018-02-23 21:06:59.285693,rejected,155964,0,18,341,35:05.5,00:05.3,completed,155964,2018-02-21 19:00:00,2018-02-23 20:00:00
2,6722,2096,2017-10-24 16:00:15.919072,2017-11-14 00:43:34.942621,closed,150635,0,14,392,26:00.1,00:01.0,completed,150635,2018-04-04 14:00:00,2018-04-11 14:30:00
3,3738,2096,2016-08-25 16:10:45.696811,2017-01-13 11:01:39.364351,closed,72364,1,14,392,26:00.1,00:01.0,completed,150635,2018-04-04 14:00:00,2018-04-11 14:30:00
4,4014,2096,2016-10-06 17:25:17.585405,2017-05-08 14:05:17.489132,completed,72364,1,14,392,26:00.1,00:01.0,completed,150635,2018-04-04 14:00:00,2018-04-11 14:30:00


In [560]:
rows_drop = project_session.index[project_session.created_at_x < project_session.scheduled_for]
project_session.drop(rows_drop, axis=0, inplace=True)

   \- Add to dataFrame project one column 'session_cnt' where session_cnt is 0 if there's no session associated with the project. Otherwise, session_cnt_nz.values are assigned to the project session_cnt column

In [561]:
session_cnt_nz = project_session.groupby('id_x')['id_y'].count()

In [602]:
project.loc[project.id.isin(session_cnt_nz.index), 'session_cnt'] = session_cnt_nz.values
project.loc[project.session_cnt.isnull(), 'session_cnt'] = 0

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
11544    0.0
11545    0.0
11546    0.0
11547    0.0
11548    0.0
Name: session_cnt, Length: 11464, dtype: float64

   \- How many sessions hold before starting a project?
   <br>\- How many projects didn't have sessions before starting the project?

In [620]:
columns = ['id', 'organization_id', 'project_category_id', 'session_cnt', 'state', 
         'project_inquiries_count', 'created_at']
prjWithSessions = project[columns].loc[project.session_cnt>0].sort_values(by='session_cnt', ascending=False)
project.pivot_table(values='id', index='session_cnt', columns='project_category_id',
                           aggfunc='count', fill_value=0, margins=True)
#prjWithSessions.pivot_table(values='id', index='session_cnt', columns='project_category_id',
 #                          aggfunc='count', fill_value=0, margins=True)

project_category_id,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,20,21,All
session_cnt,,,,,,,,,,,,,,,,,,,,,
0.0,724,1121,136,250,222,246,314,312,180,1473,...,621,1408,456,499,212,517,880,979,146,11464
1.0,5,7,2,0,0,3,0,1,3,17,...,0,5,3,2,5,6,5,8,0,76
2.0,0,0,0,0,0,0,0,0,1,1,...,0,0,1,0,1,0,0,3,0,8
3.0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
All,729,1128,138,250,222,249,314,313,184,1492,...,621,1413,460,501,218,523,885,990,146,11549


   \- Project states distribution for non-session projects?
   <br>\- Project states distribution for projects with sessions?

In [621]:
project.loc[project.session_cnt==0].pivot_table(values='id', index='state', columns='project_category_id',
                                               aggfunc='count', fill_value=0, margins=True)

project_category_id,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,20,21,All
state,,,,,,,,,,,,,,,,,,,,,
admin_close,2,2,0,0,0,3,1,3,0,6,...,3,5,3,2,1,4,2,4,0,43
applied,6,8,3,7,1,2,1,5,2,11,...,9,12,4,4,0,4,4,6,3,105
archived,9,5,1,7,2,1,3,3,3,24,...,12,15,7,6,6,6,9,17,0,148
closed,313,427,36,55,75,69,85,101,56,563,...,342,556,112,246,68,170,342,435,64,4365
completed,209,371,53,60,84,100,148,100,57,537,...,124,619,230,145,69,252,268,309,29,4075
deactivated,12,18,2,8,6,7,11,5,3,34,...,12,17,12,8,11,10,20,16,3,223
draft,7,10,2,3,2,2,3,4,1,9,...,4,12,3,3,2,2,15,14,2,110
matched,33,33,11,22,7,18,18,12,8,69,...,15,53,24,18,19,36,45,31,9,516
published,20,49,4,9,4,19,10,13,5,61,...,32,33,11,16,5,4,40,24,6,389


In [622]:
project.loc[project.session_cnt>0].pivot_table(values='id', index='state', columns='project_category_id',
                                              aggfunc='count', fill_value=0, margins=True)

project_category_id,1,2,3,6,8,9,10,11,14,15,16,17,18,19,20,All
state,,,,,,,,,,,,,,,,
admin_close,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
applied,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
archived,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,2
closed,1,2,0,0,0,0,9,2,4,0,0,3,2,0,4,27
completed,2,2,1,1,0,0,8,0,1,2,0,1,2,3,3,26
deactivated,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,2
draft,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
matched,1,2,0,0,1,0,0,2,0,1,0,1,2,1,2,13
published,0,0,0,1,0,2,1,1,0,0,2,0,0,1,1,9


- Data Analysis with Visualization Tools

   \- DataFrame - project

In [629]:
project.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11549 entries, 0 to 11548
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   id                       11549 non-null  int64         
 1   organization_id          11549 non-null  int64         
 2   created_at               11549 non-null  datetime64[ns]
 3   updated_at               11549 non-null  datetime64[ns]
 4   state                    11549 non-null  object        
 5   user_id                  11549 non-null  int64         
 6   project_inquiries_count  11549 non-null  int64         
 7   project_category_id      11549 non-null  int8          
 8   session_cnt              11549 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(4), int8(1), object(1)
memory usage: 733.2+ KB


In [ ]:
fig, ax = plt.subplots(1,1)
prj_crs = project.crosstab(project['state'], project['project_category_id'])
prj_crs